In [1]:
import datajoint as dj
dj.config.load("dj_local_conf.json")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from scipy.ndimage import gaussian_filter1d

from spyglass.position.position_merge import PositionOutput
import spyglass.lfp as lfp
from spyglass.lfp.analysis.v1 import lfp_band

/home/dmrodriguez/miniforge3/envs/mPFC-opto/lib/python3.10/site-packages/datajoint/plugin.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2026-01-19 11:46:42,343][INFO]: DataJoint is configured from /home/dmrodriguez/mPFC-opto/notebooks/dj_local_conf.json
[2026-01-19 11:46:43,534][INFO]: DataJoint 0.14.6 connected to denissemorales@lmf-db.cin.ucsf.edu:3306


In [2]:
from sleep.sleep_table_dmr import SleepScoringParams, SleepScoringSelection, SleepScoring

# define nwb_file, interval list, filter names, and lfp electrodes

nwb_file_name = 'Jacob20250617_.nwb'
lfp_electrode_group_name = 'Left and Right CA1 Hippocampus'
interval_list_name = '03_s2'

lfp_filter = "LFP 0-400 Hz"
delta_filter = 'Delta 0.5-4 Hz'
theta_filter = 'Theta 5-11 Hz'
emg_filter = 'EMG 300-600'

sampling_rate = 1000


In [5]:
# Here, method can be hierarchical, GMM, or Kmeans. Use hierarchical just means
# you distinguish sleep from wake first, as opposed to clustering wake/NREM/REM
# at the same time

SleepScoringParams()

sleep_scoring_params_name,method Classification method,use_hierarchical Use two-stage classification,use_pss Include power spectrum slope if available,power_smoothing Gaussian smoothing sigma for power (seconds),speed_smoothing Gaussian smoothing sigma for speed (seconds),apply_constraints Apply physiological constraints,rem_cannot_follow_wake REM cannot directly follow WAKE,constraint_max_iterations Max iterations for constraint enforcement,min_duration Minimum state bout duration (seconds),speed_threshold Speed threshold for wake detection (cm/s),use_speed_for_wake Use speed instead of EMG for wake detection
hierarchical,hierarchical,1,0,0.5,0.5,1,1,15,5.0,3.0,1


In [7]:
lfp_s_key = {
    "nwb_file_name": nwb_file_name,
    "lfp_electrode_group_name": lfp_electrode_group_name,
    "target_interval_list_name": interval_list_name,
    "filter_name": lfp_filter,
    "target_sampling_rate": sampling_rate
}

lfp_merge_id = (lfp.LFPOutput.LFPV1() & lfp_s_key).fetch1("merge_id")


theta_merge_id = (lfp_band.LFPBandV1 & {'filter_name': theta_filter,
                                   'nwb_file_name': nwb_file_name,
                                  'target_interval_list_name': interval_list_name}).fetch1('lfp_merge_id')
delta_merge_id = (lfp_band.LFPBandV1 & {'filter_name': delta_filter,
                                   'nwb_file_name': nwb_file_name,
                                  'target_interval_list_name': interval_list_name}).fetch1('lfp_merge_id')


In [15]:
lfp_band.LFPBandV1() & {
                        'filter_name': emg_filter}

lfp_merge_id,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,nwb_file_name name of the NWB file,target_interval_list_name descriptive name of this interval list,lfp_band_sampling_rate the sampling rate for this band,analysis_file_name name of the file,interval_list_name descriptive name of this interval list,lfp_band_object_id the NWB object ID for loading this object from the file


In [10]:
pos_key = (PositionOutput.TrodesPosV1() & {'nwb_file_name': nwb_file_name,
                             'interval_list_name': "pos 0 valid times",
                             'trodes_pos_params_name': 'single_led'}).fetch1("KEY")
pos_merge_id = pos_key['merge_id']

In [17]:
key = [{'sleep_scoring_params_name': 'hierarchical',
       'lfp_merge_id': lfp_merge_id,
       'nwb_file_name': nwb_file_name,
        'filter_sampling_rate': sampling_rate,
        'target_interval_list_name': '03_s2',
        'pos_merge_id':pos_key['merge_id'],
        'theta_filter_name': theta_filter,
        'delta_filter_name': delta_filter,
        'emg_filter_name': None,
        'pss_filter_name': None,
        'emg_merge_id': None,
        'pss_merge_id': None
}]

In [ ]:
position_df = (PositionOutput & pos_key).fetch1_dataframe()


In [12]:
position_df

,video_frame_ind,position_x,position_y,orientation,velocity_x,velocity_y,speed
time,,,,,,,
1.750178e+09,0,40.29,23.052,NaN,6.216477e-15,-1.320098e-14,1.459145e-14
1.750178e+09,1,40.29,23.052,NaN,9.169628e-15,-1.470617e-14,1.733071e-14
1.750178e+09,2,40.29,23.052,NaN,1.210399e-14,-1.470617e-14,1.904673e-14
1.750178e+09,3,40.29,23.052,NaN,1.429797e-14,-1.320098e-14,1.946016e-14
1.750178e+09,4,40.29,23.052,NaN,1.511436e-14,-1.063681e-14,1.848204e-14
...,...,...,...,...,...,...,...
1.750179e+09,54780,40.29,23.052,NaN,-8.865047e-15,-6.051994e-15,1.073386e-14
1.750179e+09,54781,40.29,23.052,NaN,-6.093618e-15,-4.584814e-15,7.625791e-15
1.750179e+09,54782,40.29,23.052,NaN,-3.727273e-15,-3.108238e-15,4.853217e-15


In [22]:
key

[{'sleep_scoring_params_name': 'hierarchical',
  'lfp_merge_id': UUID('f8b497a4-75a1-dafe-2c37-da68ad87f0af'),
  'nwb_file_name': 'Jacob20250617_.nwb',
  'filter_sampling_rate': 1000,
  'target_interval_list_name': '03_s2',
  'pos_merge_id': UUID('aee5cc01-03a0-f552-5c4f-4b9cb28b6314'),
  'theta_filter_name': 'Theta 5-11 Hz',
  'delta_filter_name': 'Delta 0.5-4 Hz',
  'emg_filter_name': None,
  'pss_filter_name': None,
  'emg_merge_id': None,
  'pss_merge_id': None}]

In [6]:
SleepScoringSelection().drop()

[2026-01-18 12:30:32,100][INFO]: `denissemorales_sleepscoring`.`sleep_scoring_selection` (2 tuples)
[2026-01-18 12:30:32,221][INFO]: `denissemorales_sleepscoring`.`__sleep_scoring` (2 tuples)
[2026-01-18 12:30:33,837][INFO]: Dropped table `denissemorales_sleepscoring`.`__sleep_scoring`
[2026-01-18 12:30:33,979][INFO]: Dropped table `denissemorales_sleepscoring`.`sleep_scoring_selection`
[2026-01-18 12:30:34,010][INFO]: Tables dropped. Restart kernel.


In [19]:
SleepScoringSelection().insert(key, skip_duplicates = True)

In [4]:
SleepScoring()

sleep_scoring_params_name,target_interval_list_name,nwb_file_name,theta_filter_name,delta_filter_name,"lfp_merge_id Base LFP merge (optional, if needed)","state_labels Array of state labels (0=NREM, 1=REM, 2=WAKE)",timestamps Timestamps for state labels,nrem_duration Total NREM duration (seconds),rem_duration Total REM duration (seconds),wake_duration Total WAKE duration (seconds),nrem_percentage Percentage of time in NREM,rem_percentage Percentage of time in REM,wake_percentage Percentage of time in WAKE,analysis_file_name name of the file,trial_object_id


In [4]:
SleepScoring().drop()

[2026-01-19 11:46:35,502][INFO]: `denissemorales_sleepscoring`.`__sleep_scoring` (0 tuples)
[2026-01-19 11:46:37,072][INFO]: Dropped table `denissemorales_sleepscoring`.`__sleep_scoring`
[2026-01-19 11:46:37,099][INFO]: Tables dropped. Restart kernel.


In [3]:
SleepScoring.populate()

/home/dmrodriguez/miniforge3/envs/spyglass/lib/python3.9/site-packages/hdmf/spec/namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
ndx-franklab-novela - cached version: 0.2.0, loaded version: 0.2.3
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)
/home/dmrodriguez/miniforge3/envs/spyglass/lib/python3.9/site-packages/hdmf/spec/namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
ndx-franklab-novela - cached version: 0.2.0, loaded version: 0.2.3
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignore

Wake detection using head speed: 0 epochs WAKE
NREM epochs: 1827817, REM epochs: 1
REM fallback applied: 169465 epochs set to REM


{'success_count': 1, 'error_list': []}

In [2]:
import spyglass.common as sgc
sgc.IntervalList().check_threads()

,Locked,Name,Time (s),Process,State


In [3]:
SleepScoring().drop()

NameError: name 'SleepScoring' is not defined

In [16]:
import numpy as np

sws_intervals = np.array([[1.62230601e+09, 1.62230607e+09],
                          [1.62230628e+09, 1.62230629e+09],
                          [1.62230631e+09, 1.62230632e+09],
                          [1.62230636e+09, 1.62230643e+09],
                          [1.62230646e+09, 1.62230656e+09],
                          [1.62230657e+09, 1.62230664e+09],
                          [1.62230694e+09, 1.62230697e+09],
                          [1.62230698e+09, 1.62230735e+09],
                          [1.62230738e+09, 1.62230750e+09],
                          [1.62230753e+09, 1.62230784e+09],
                          [1.62230791e+09, 1.62230809e+09],
                          [1.62230810e+09, 1.62230820e+09]])

rem_intervals = np.array([[1.62230629e+09, 1.62230631e+09],
                          [1.62230632e+09, 1.62230633e+09],
                          [1.62230635e+09, 1.62230636e+09],
                          [1.62230656e+09, 1.62230657e+09],
                          [1.62230676e+09, 1.62230677e+09],
                          [1.62230688e+09, 1.62230688e+09],
                          [1.62230735e+09, 1.62230738e+09],
                          [1.62230750e+09, 1.62230753e+09],
                          [1.62230784e+09, 1.62230791e+09],
                          [1.62230820e+09, 1.62230836e+09],
                          [1.62230843e+09, 1.62230844e+09]])

# Calculate total durations
total_nrem = np.sum(sws_intervals[:,1] - sws_intervals[:,0])
total_rem  = np.sum(rem_intervals[:,1] - rem_intervals[:,0])

# REM/NREM duration ratio
rem_to_nrem_ratio = total_rem / total_nrem

total_nrem, total_rem, rem_to_nrem_ratio


(1430.0, 360.0, 0.2517482517482518)

In [5]:
SleepScoring() & {'nwb_file_name': 'J1620210529_.nwb'}

NameError: name 'SleepScoring' is not defined

In [10]:
(SleepScoring() & {'nwb_file_name': 'J1620210529_.nwb'}).fetch_nrem_times()

array([[1.62230589e+09, 1.62230590e+09],
       [1.62230593e+09, 1.62230594e+09],
       [1.62230595e+09, 1.62230622e+09],
       [1.62230624e+09, 1.62230678e+09],
       [1.62230679e+09, 1.62230680e+09],
       [1.62230682e+09, 1.62230683e+09],
       [1.62230685e+09, 1.62230690e+09],
       [1.62230690e+09, 1.62230847e+09],
       [1.62230851e+09, 1.62230851e+09],
       [1.62230856e+09, 1.62230857e+09]])

In [11]:
[[1.62230601e+09, 1.62230607e+09],
              [1.62230628e+09, 1.62230629e+09],
              [1.62230631e+09, 1.62230632e+09],
              [1.62230636e+09, 1.62230643e+09],
              [1.62230646e+09, 1.62230656e+09],
              [1.62230657e+09, 1.62230664e+09],
              [1.62230694e+09, 1.62230697e+09],
              [1.62230698e+09, 1.62230735e+09],
              [1.62230738e+09, 1.62230750e+09],
              [1.62230753e+09, 1.62230784e+09],
              [1.62230791e+09, 1.62230809e+09],
              [1.62230810e+09, 1.62230820e+09]]

[[1622306010.0, 1622306070.0],
 [1622306280.0, 1622306290.0],
 [1622306310.0, 1622306320.0],
 [1622306360.0, 1622306430.0],
 [1622306460.0, 1622306560.0],
 [1622306570.0, 1622306640.0],
 [1622306940.0, 1622306970.0],
 [1622306980.0, 1622307350.0],
 [1622307380.0, 1622307500.0],
 [1622307530.0, 1622307840.0],
 [1622307910.0, 1622308090.0],
 [1622308100.0, 1622308200.0]]

In [8]:
import spyglass.common as sgc
# interval_list_name = 'pos 2 valid times'
# nwb_file_name = 'J1620210529_.nwb'

interval_list_name = '03_s2'
nwb_file_name = 'Jacob20250617_.nwb'

interval_list_name = f"{interval_list_name}_rem"
(sgc.IntervalList() & {
    'nwb_file_name': nwb_file_name,
    'interval_list_name': interval_list_name
}).delete()

[2026-01-15 14:05:14,195][WARNING]: Reconnecting to MySQL server.
[14:05:14][WARNING] Spyglass: Spawning tables for decoding_clusterless
[14:05:14][WARNING] Spyglass: Spawning tables for decoding_sortedspikes
[14:05:14][WARNING] Spyglass: Skipping unimported: `jguidera_maze`.`__annotated_universal_track_graph`
[14:05:14][WARNING] Spyglass: Spawning tables for sharing_kachery
[14:05:14][WARNING] Spyglass: Spawning tables for spikesorting_curation
[14:05:14][WARNING] Spyglass: Spawning tables for spikesorting_recording
[14:05:15][WARNING] Spyglass: Skipping unimported: `sc4712_lfp_v1`.`l_f_p_selection_sharon`
[14:05:15][WARNING] Spyglass: Spawning tables for spikesorting_v1_recording
[14:05:15][WARNING] Spyglass: Spawning tables for spikesorting_v1_sorting
[14:05:15][WARNING] Spyglass: Spawning tables for spikesorting_v1_curation
[14:05:15][WARNING] Spyglass: Spawning tables for spikesorting_v1_metric_curation
[14:05:15][WARNING] Spyglass: Spawning tables for spikesorting_imported
[14:05

LostConnectionError: Connection was lost during a transaction.

In [5]:
nwb_file_name = 'J1620210529_.nwb'
lfp_electrode_group_name = 'all_tets_J16'
interval_list_name = 'pos 2 valid times'

lfp_filter = "LFP 0-400 Hz"
delta_filter = 'Delta 0.5-4 Hz'
theta_filter = 'Theta 5-11 Hz'
emg_filter = 'EMG 300-6/00 DMR'
sampling_rate = 1000

lfp_key = '2c3a6142-c62c-0ba8-2b35-d083790590e5'

lfp_deltaband_key = (
    lfp_band.LFPBandSelection
    & {
        "lfp_merge_id": lfp_key,
        "filter_name": delta_filter,
        "lfp_band_sampling_rate": 1000,
        'target_interval_list_name': interval_list_name
    }
).fetch1("KEY")

lfp_thetaband_key = (
    lfp_band.LFPBandSelection
    & {
        "lfp_merge_id": lfp_key,
        "filter_name": theta_filter,
        "lfp_band_sampling_rate": 1000,
        'target_interval_list_name': interval_list_name
    }
).fetch1("KEY")

pos_key = (PositionOutput.TrodesPosV1() & {'nwb_file_name': nwb_file_name,
                             'interval_list_name': 'pos 2 valid times'}
           ).fetch1("KEY")


In [6]:
key = [{'sleep_scoring_params_name': 'hierarchical',
       'lfp_merge_id': lfp_key,
       'nwb_file_name': nwb_file_name,
        'filter_sampling_rate': sampling_rate,
        'target_interval_list_name': interval_list_name,
        'pos_merge_id':pos_key['merge_id'],
        'theta_filter_name': theta_filter,
        'delta_filter_name': delta_filter,
        'emg_filter_name': None,
        'pss_filter_name': None,
        'emg_merge_id': None,
        'pss_merge_id': None
}]

In [7]:
SleepScoringSelection().insert(key, skip_duplicates = True)

In [8]:
SleepScoring.populate()

/home/dmrodriguez/miniforge3/envs/spyglass/lib/python3.9/site-packages/hdmf/spec/namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.8.0, loaded version: 2.7.0
ndx-franklab-novela - cached version: 0.1.0, loaded version: 0.2.3
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)
/home/dmrodriguez/miniforge3/envs/spyglass/lib/python3.9/site-packages/hdmf/spec/namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
ndx-franklab-novela - cached version: 0.2.0, loaded version: 0.2.3
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are c

Using head speed for wake: 291124 wake epochs
REM fallback applied: 230072 epochs set to REM


ValueError: Found 2 rows with existing names, but different times:[{'nwb_file_name': 'J1620210529_.nwb', 'interval_list_name': 'pos 2 valid times_nrem', 'valid_times': array([[1.62230593e+09, 1.62230594e+09],
       [1.62230596e+09, 1.62230622e+09],
       [1.62230624e+09, 1.62230678e+09],
       [1.62230682e+09, 1.62230683e+09],
       [1.62230685e+09, 1.62230690e+09],
       [1.62230690e+09, 1.62230847e+09],
       [1.62230856e+09, 1.62230857e+09]]), 'pipeline': '`denissemorales_sleepscoring`.`__sleep_scoring`'}, {'nwb_file_name': 'J1620210529_.nwb', 'interval_list_name': 'pos 2 valid times_wake', 'valid_times': array([[1.62230587e+09, 1.62230593e+09],
       [1.62230594e+09, 1.62230596e+09],
       [1.62230622e+09, 1.62230624e+09],
       [1.62230678e+09, 1.62230682e+09],
       [1.62230683e+09, 1.62230685e+09],
       [1.62230690e+09, 1.62230690e+09],
       [1.62230847e+09, 1.62230856e+09]]), 'pipeline': '`denissemorales_sleepscoring`.`__sleep_scoring`'}]

In [21]:
(SleepScoring() & key).fetch_nrem_times()

array([[1.62230589e+09, 1.62230590e+09],
       [1.62230593e+09, 1.62230594e+09],
       [1.62230595e+09, 1.62230622e+09],
       [1.62230624e+09, 1.62230678e+09],
       [1.62230679e+09, 1.62230680e+09],
       [1.62230682e+09, 1.62230683e+09],
       [1.62230685e+09, 1.62230690e+09],
       [1.62230690e+09, 1.62230847e+09],
       [1.62230851e+09, 1.62230851e+09],
       [1.62230856e+09, 1.62230857e+09]])

In [22]:
SleepScoring() & key

sleep_scoring_params_name,target_interval_list_name,nwb_file_name,theta_filter_name,delta_filter_name,"lfp_merge_id Base LFP merge (optional, if needed)",nrem_interval_list_name descriptive name of this interval list,rem_interval_list_name descriptive name of this interval list,wake_interval_list_name descriptive name of this interval list,"state_labels Array of state labels (0=NREM, 1=REM, 2=WAKE)",timestamps Timestamps for state labels,nrem_duration Total NREM duration (seconds),rem_duration Total REM duration (seconds),wake_duration Total WAKE duration (seconds),nrem_percentage Percentage of time in NREM,rem_percentage Percentage of time in REM,wake_percentage Percentage of time in WAKE
hierarchical,pos 2 valid times,J1620210529_.nwb,Theta 5-11 Hz,Delta 0.5-4 Hz,2c3a6142-c62c-0ba8-2b35-d083790590e5,pos 2 valid times_nrem,pos 2 valid times_rem,pos 2 valid times_wake,=BLOB=,=BLOB=,2489.03,0.0,210.801,92.1914,0.0,7.80787
